In [ ]:
# %%
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from keras import layers, models, regularizers, callbacks
from keras.api.optimizers import Adam
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from pytorch_tabnet.tab_model import TabNetRegressor


# %%
# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the dataset
df = pd.read_csv("./lucas_pre.csv")

# Display basic information
print(f"Dataset shape: {df.shape}")

# Identify column types
target = "pH_H2O"
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if target in numeric_cols:
    numeric_cols.remove(target)

categorical_cols = ["Depth", "LC", "LU", "USDA", "ISSS", "NUTS_0", "LC0_Desc", "LC1_Desc", "LU1_Desc"]
# Filter to only keep categorical columns that exist in the dataset
categorical_cols = [col for col in categorical_cols if col in df.columns]

# Remove columns with too many unique values or too many missing values
filtered_cat_cols = []
for col in categorical_cols:
    if col in df.columns:
        if df[col].nunique() < 30 and df[col].isna().sum() / len(df) < 0.3:
            filtered_cat_cols.append(col)

print(f"Using {len(numeric_cols)} numeric columns and {len(filtered_cat_cols)} categorical columns")

# Drop rows where target is missing
df = df.dropna(subset=[target])

# Feature engineering - create new features
if "EC" in df.columns and "OC" in df.columns:
    df["EC_OC_ratio"] = df["EC"] / df["OC"].replace(0, 0.001)

if "Clay" in df.columns and "Sand" in df.columns:
    df["Clay_Sand_ratio"] = df["Clay"] / df["Sand"].replace(0, 0.001)

if "N" in df.columns and "P" in df.columns and "K" in df.columns:
    # NPK balance is important for soil chemistry
    if df["N"].notnull().sum() > 0 and df["P"].notnull().sum() > 0 and df["K"].notnull().sum() > 0:
        df["NPK_sum"] = df["N"] + df["P"] + df["K"]

if "CaCO3" in df.columns:
    # Soil pH is strongly related to CaCO3
    df["log_CaCO3"] = np.log1p(df["CaCO3"])

if "OC" in df.columns and "N" in df.columns:
    # C:N ratio is important for soil biology
    df["CN_ratio"] = df["OC"] / df["N"].replace(0, 0.001)

if "Clay" in df.columns and "OC" in df.columns:
    # Clay-organic matter interactions affect pH
    df["Clay_OC_interaction"] = df["Clay"] * df["OC"]

# Extract target values
y = df[target].values

# Update numeric columns after adding engineered features
numeric_cols = [col for col in df.select_dtypes(include=[np.number]).columns.tolist() if col != target]

# Create preprocessing pipeline
numeric_transformer = Pipeline(
    steps=[
        ("imputer", KNNImputer(n_neighbors=5)),
        ("power", PowerTransformer(method="yeo-johnson")),  # Better than StandardScaler for skewed data
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))])

# Combine all transformers
preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_cols), ("cat", categorical_transformer, filtered_cat_cols)]
)

# Prepare features
X = df[numeric_cols + filtered_cat_cols]

# Split data before preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


In [ ]:
# Apply preprocessing
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(f"Processed feature shape: {X_train_processed.shape}")

# Split training data to create a validation set
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train_processed, y_train, test_size=0.2, random_state=42
)

# Convert to numpy arrays if they aren't already
X_train_np = X_train_split
X_val_np = X_val
X_test_np = X_test_processed

# Reshape target variables to 2D for TabNet
y_train_np = y_train_split.reshape(-1, 1)
y_val_np = y_val.reshape(-1, 1)
y_test_np = y_test.reshape(-1, 1)

# Define TabNet model with hyperparameters
tabnet_model = TabNetRegressor(
    n_d=64,  # Width of the decision prediction layer
    n_a=64,  # Width of the attention embedding for each mask
    n_steps=5,  # Number of steps in the architecture (feature selection steps)
    gamma=1.5,  # This is the coefficient for feature reusage in the masks
    n_independent=2,  # Number of independent GLU layers at each step
    n_shared=2,  # Number of shared GLU layers at each step
    cat_idxs=[],  # No categorical indices since we've already preprocessed
    cat_dims=[],  # No categorical dimensions
    cat_emb_dim=[],  # No categorical embedding dimensions
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.01),
    scheduler_params=dict(
        mode="min",
        # patience=10,
        min_lr=1e-5,
        factor=0.5,
    ),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    mask_type="entmax",  # "sparsemax" or "entmax" for more flexibility in feature selection
    # lambda_sparse=1e-4,  # Sparsity regularization
    # max_epochs=200,  # Maximum number of epochs (with early stopping)
    # patience=20,  # Early stopping patience
    # batch_size=1024,  # Larger batch size for faster training
    # virtual_batch_size=128,  # Size of the mini-batches used for "Ghost Batch Normalization"
    # num_workers=0,  # Number of workers for data loading
    # drop_last=False,  # Whether to drop the last incomplete batch
    verbose=1,  # Verbosity level
)

# Fit the model
tabnet_model.fit(
    X_train_np,
    y_train_np,
    eval_set=[(X_train_np, y_train_np), (X_val_np, y_val_np), (X_test_np, y_test_np)],
    eval_name=["train", "valid", "test"],
    eval_metric=["mae"],
    patience=20,
    max_epochs=200,
    batch_size=64,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
)

# Predict on test data
y_pred = tabnet_model.predict(X_test_np)

# Ensure predictions match the target shape for evaluation
y_test_flat = y_test_np.ravel()
y_pred_flat = y_pred.ravel()

# Evaluate the model
mae = mean_absolute_error(y_test_flat, y_pred_flat)
r2 = r2_score(y_test_flat, y_pred_flat)

print(f"Test Mean Absolute Error: {mae:.4f}")
print(f"R² Score: {r2:.4f}")

# Get feature importances (one of TabNet's advantages)
feature_importances = tabnet_model.feature_importances_

# If you have the original feature names
if "numeric_cols" in locals() and "filtered_cat_cols" in locals():
    feature_names = numeric_cols + filtered_cat_cols
    # Create a dictionary of feature importances
    importance_dict = dict(zip(feature_names, feature_importances))
    # Sort by importance
    sorted_importances = {k: v for k, v in sorted(importance_dict.items(), key=lambda item: item[1], reverse=True)}

    print("\nFeature Importances:")
    for feature, importance in sorted_importances.items():
        print(f"{feature}: {importance:.4f}")

# Plot training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(tabnet_model.history["loss"], label="train")
plt.plot(tabnet_model.history["val_loss"], label="validation")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

# For comparison with the neural network
print("\nComparison:")
print(f"TabNet MAE: {mae:.4f}, R²: {r2:.4f}")
print(f"Neural Network MAE: {mae:.4f}, R²: {r2:.4f}")  # Use your NN results here

# You can also get the masks used by TabNet for interpretability
# This shows which features were used at each decision step
masks = tabnet_model.explain(X_test_np)